# AutoTSI Postprocessing Template

## Stationary Sampling

In [ ]:
from source import *

step_start = 0  # smallest angle (step number) on LiDAR scanner to start interpreting
step_end = 1080  # largest angle (step number) on LiDAR scanner to stop interpreting
distance_maximum = 3000  # maximum distance from LiDAR to interpret from (cm)
ratio_minimum = 5 * 1440  # minimum ratio of steps/radians to consider a 'tree' (i.e. how flat a measured object can be
ratio_maximum = 150 * 1500  # maximum ratio of steps/radians to consider a 'tree' (i.e. how steep an object can be)
limit_noise = 35  # max distance the center of an object can deviate from each time step
diameter_minimum = 2 * 25.4  # minimum allowable diameter of a tree to consider
diameter_maximum = 60 * 25.4  # maximum allowable diameter of a tree to consider
limit_tracking_error = 0.05  # % difference between measurements to consider same tree for tracking

In [ ]:
# Process stationary sampling inventory
stat_file = 'path/to/stationary/test/file.txt' # output of stationary_sampling.py
telem_log = 'path/to/telem/file.txt'  # telemetry log from mission planner
shp_reference = 'path/to/shapefile.shp'  # shapefile to plot over

tree_DF_verify, sample_areas_GDF = dbh_pipeline_stationary(
    stat_file,
    telem_log,
    distance_maximum,
    step_start,
    step_end
)

tree_GDF_resolved = make_geograph(
    sample_areas_GDF,
    tree_DF_verify,
    shp_reference,
    1
)

tree_GDF_resolved, basal_area_per_hectare = calculate_basal_area(tree_GDF_resolved)

## Non-stationary Sampling

In [ ]:
# Process generic inventory, **Requires high-accuracy GPS measurements!
scanFile = 'path/to/scan/file.txt'  # raw output stream from LiDAR
telem_log = 'path/to/telem/file.txt'  # telemetry log from mission planner
xls_raw = 'path/to/raw/tree/spreadsheet.xlsx'  # spreadsheet to write raw measurments
shp_reference = 'path/to/shapefile.shp'  # shapefile to plot over
level_tracking = 0  # tracking level, 0 for least strict, 3 for most strict
xls_resolution = 'path/to/resolved/tree/spreadsheet.xlsx'  # spreadsheet to write resolved measurements

In [ ]:
# Run standard pipeline to create raw measurement dataframe
tree_DF_raw = dbh_pipeline(
    scanFile,
    telem_log,
    xls_raw,
    level_tracking,
    step_start,
    step_end,
    distance_maximum,
    ratio_minimum,
    ratio_maximum,
    limit_noise,
    diameter_minimum, 
    diameter_maximum,
    limit_tracking_error
)

xls_resolution = 'path/to/resolved/tree/spreadsheet.xlsx'  # spreadsheet to write resolved measurements
threshold_intersection_filter = 50  # minimum number of overlapping measurements to consider a tree in that location

tree_DF_resolved = resolve_tree_data(
    tree_DF_raw,
    xls_resolution, 
    threshold_intersection_filter
)  # resolve trees

tree_GDF_resolved = make_geograph(
    sample_areas_GDF, 
    tree_DF_resolved,
    shp_reference,
    1
) # convert to geodataframe

tree_GDF_resolved, basal_area_per_hectare = calculate_basal_area(tree_GDF_resolved)  # calculate basal area